#Proyecto de Análisis Númerico - Interpolación de Curvas Paramétricas
##Presentado por: David Molano y Dorian Moreno
###Pontificia Universidad Javeriana 2018

####Objetivo
En este proyecto se prentende crear una función que será parte de una libreria con utilidades de análisis númerico para el lenguaje de programación R. La función que se implementará permite realizar interpolación de curvas descritas de forma paramétrica.

####Problema
La función que se quiere implementar debe recibir un conjunto de coordenadas ${(x,y)}$ y los valores de una tercera variable ${t}$ llamada parámetro. Dadas estas entradas la salida esperad son dos polinomios ${x=f(t)}$ y ${y=g(t)}$ que interpolan dichas coordenadas en función del parámetro.
A continuación se hace un respaso teorico de las curvas paramétricas y del método de interpolación que se va a utilizar para calcular las salidas.

####Curvas Paramétricas
>Las curvas paramétricas permiten describir curvas que no pueden ser representadas por una función de la forma ${y=f(x)}$. Pero teniendo un conjunto de coordenadas $x$ , $y$ que sean funciones del tiempo $t$ se pueden escribir de la forma ${x=f(t)}$ y ${y=g(t)}$. Asi pues, dadas dos funciones $x$ y $y$ de un una tercera variable $t$ llamada parámetro mediante las ecuaciones anteriormente descritas cada valor de $t$ determina un punto ${(x,y)}$ que se puede representar en un sistema coordenado. Cuando _t_ varia ${(x,y)=(f(t),g(t))}$ varía y traza una curva $C$, a la cual se le llama curva paramétrica. ( _Tomado de "Cálculo de Una variable". Stewart. 2008._ )

####Interpolación polinómica de Lagrange
>Con la intención de obtener los polinomios de la forma ${x=f(t)}$ y ${y=g(t)}$ se utilizará la interpolación de Lagrange. Este método permite encontrar el polinomio interpolante de un conjunto de puntos dado. 
Se define formalmente asi:
Dado un conjunto de _k_ + 1 puntos ${(x_{0},y_{0}),....,(x_{k},y_{k})}$ donde todos los ${x_{j}}$ se asumen distintos, el polinomio interpolador de Lagrange es la combinación lineal ${L(x)=\sum_{j=0}^{k}y_{j}\ell_{j}(x)}$ de bases polinómicas de Lagrange ${\ell_{j}(x)=\prod_{i=0,i\neq j}^{k}\frac{x-x_{i}}{x_{j}-x_{i}}=\frac{x-x_{0}}{x_{j}-x_{0}}...\frac{x-x_{j-1}}{x_{j}-x_{j-1}}...\frac{x-x_{k}}{x_{j}-x_{k}}}$. ( _Tomado de "Análisis Númerico". Burden. 2011._ )

####Solución
En su libro de Análisis Númerico, Burden propone que para aproximar curvas parámetricas se puede utilizar el método de Lagrange. Para esto, el problema se resume a realizar dos interpolaciones. Para esto se deben construir dos conjuntos de puntos a partir de los dados. El primer conjunto  es el que produce el polinomio ${x=f(t)}$ y el conjunto debe ser ${(t_{0},x_{0}),...,(t_{k},x_{k})}$. El segundo conjunto que produce el polinomio ${y=g(t)}$ es ${(t_{0},y_{0}),...,(t_{k},y_{k})}$. De esta manera se puede obtener una curva definida de forma paramétrica para los puntos dados.

In [1]:
as.function <- function(formula) {
  function(x) eval(expr = parse(text=formula), list(x))
}

lagrange_constructor <- function(t, cor) {
  n = length(t)
  pol <- seq(from = 0, to = 0, len = n)
  for(i in (1:n))
  {
    polytemp <- c(1)
    factor = 1
    for(j in (1:n))
    {
      if(j == i)
        next
      size = length(polytemp)
      new <- seq(from = 0, to = 0, len = size)
      for(k in (1:size))
      {
        temp = polytemp[k]
        otro = -t[j]
        new[k] = new[k] + temp*otro
        new[k+1] = temp
      }
      factor = factor*(t[i] - t[j])
      polytemp <- new
    }
    factor = cor[i]/factor
    for(i in (0:n))
    {
      pol[i] = pol[i] + polytemp[i]*factor
    }
  }
  ##Aquí ya tenemos contruido el polinomio pol (arreglo de coeficientes) así que ahora creamos
  ## la equación eq:
  eq = ""
  if(pol[1]!=0)
    eq <- paste(pol[1], "", sep = "")
  if(pol[2]<0)
  {
    ##Negativo
    eq <- paste(eq, paste(-pol[2], "x", sep = "*"), sep = " - ")
  }
  else if(pol[2]>0)
  {
    ##Positivo
    eq <- paste(eq, paste(pol[2], "x", sep = "*"), sep = " + ")
  }
  for(i in (3:n))
  {
    if(pol[i]<0)
    {
      ##Negativo
      eq <- paste(eq, paste(-pol[i], paste("x^",i-1 , sep = ""), sep = "*"), sep = " - ")
    }
    else if(pol[i]>0)
    {
      ##Positivo
      eq <- paste(eq, paste(pol[i], paste("x^",i-1 , sep = ""), sep = "*"), sep = " + ")
    }
  }
  if(substr(eq, 1, 1) == " ")
  {
    if(eq[1]=="-")
      eq = paste("-", substr(eq, 4, nchar(eq)), sep = "")
    else
      eq = substr(eq, 4, nchar(eq))
  }
  return(eq)
}


lagrange_parametric <- function(t, x, y){
  
  x_cor = lagrange_constructor(t, x)
  y_cor = lagrange_constructor(t, y)
  return(list("fx" = as.function(x_cor), "fy" = as.function(y_cor)))
}

lagrange_evaluator <- function(t, formulas)
{
  x = formulas$fx(t)
  y = formulas$fy(t)
  return(list("x" = x, "y" = y))
}

  
t = c(0,0.25,0.5,0.75,1)
x = c(-1,0,1,0,1)
y = c(0,1,0.5,0,-1)

formulas = lagrange_parametric(t,x,y)
lagrange_evaluator(t, formulas)

$x
[1] -1.000000e+00  4.440892e-15  1.000000e+00  1.421085e-13  1.000000e+00

$y
[1]  0.000000e+00  1.000000e+00  5.000000e-01 -7.105427e-15 -1.000000e+00